In [1]:
import nltk 
from PIL import Image
import pytesseract
from wand.image import Image as Img 
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import os
import cv2
import shutil
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from collections import OrderedDict

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user.DESKTOP-3R3KUAJ\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
parent_dir = 'C:\\Users\\user.DESKTOP-3R3KUAJ\\Desktop\\Python\\nlp'

In [21]:
def remove_duplicate_sentences(file_path):
    # Read the contents of the text file
    with open(file_path, 'r') as file:
        text = file.read()

    # Split the text into sentences
    sentences = text.split('.')

    # Create a set to store unique sentences
    unique_sentences = set()
    
    # Iterate through each sentence and add it to the set
    for sentence in sentences:
        sentence = sentence.strip()  # Remove leading/trailing whitespace
        if sentence:  # Skip empty sentences
            unique_sentences.add(sentence)

    # Convert the set back to a list of sentences
    unique_sentences = list(unique_sentences)
    print(unique_sentences)
    # Write the unique sentences back to the text file
    with open(file_path, 'w') as file:
        file.write('.'.join(unique_sentences))

In [22]:
def files():
    try:
        for dir in os.listdir(parent_dir):
            try:
                shutil.rmtree(os.path.join(parent_dir, dir))
            except:
                os.remove(os.path.join(parent_dir, dir))
    except OSError:
        pass

In [23]:
def get_audio_text(start, end, file_path, file_name):
    audio_files_dir = os.path.join(parent_dir, 'converted')

    if not os.path.exists(audio_files_dir):
        os.makedirs(audio_files_dir)

    text = ''
    chunk_file = os.path.join(file_path, file_name)
    ffmpeg_extract_subclip("Servlet.mp4", start, end, targetname = chunk_file)
    video = cv2.VideoCapture(chunk_file)
    clip = mp.VideoFileClip(chunk_file)
    clip.audio.write_audiofile(os.path.join(audio_files_dir, 'converted{}.wav'.format(i+1)))
    
    r = sr.Recognizer()
    audio = sr.AudioFile(os.path.join(audio_files_dir, 'converted{}.wav'.format(i+1)))
    with audio as source:
        r.adjust_for_ambient_noise(source)  
        audio_file = r.record(source)
    try:
        result = r.recognize_google(audio_file)
    except: 
        result = 'Could not understand audio'
    diz['chunk{}'.format(i+1)]=result

    l_chunks=[diz['chunk{}'.format(i+1)] for i in range(len(diz))]
    text='\n'.join(l_chunks)

    return text

In [24]:
def get_image_text(src_vid):
    
    image_frames_dir = os.path.join(parent_dir, 'image_frames')
    
    if not os.path.exists(image_frames_dir):
        os.makedirs(image_frames_dir)

    text = ''
    index = 0
    
    frame_rate = math.floor(src_vid.get(cv2.CAP_PROP_FPS))

    while src_vid.isOpened():
        ret, frame = src_vid.read()
        if not ret:
            break

        # name each frame and save as png
        mod = index % frame_rate == 0 if index > 0 else 1
        if(mod):
            name = os.path.join(image_frames_dir, str(index) + '.png')
            print('Extracting frames...' + name)
            cv2.imwrite(name, frame)
        index = index + 1
    src_vid.release()
     
        
    for i in os.listdir(image_frames_dir):
        my_example = Image.open(os.path.join(image_frames_dir, i))
        text += pytesseract.image_to_string(my_example, lang='eng')
    return text

In [25]:
import speech_recognition as sr 
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import math

files()

chunk_files_dir = os.path.join(parent_dir, 'chunks')

video = cv2.VideoCapture('topology.mp4')
frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
frame_rate = video.get(cv2.CAP_PROP_FPS)
duration = frame_count / frame_rate
num_seconds_video = math.floor(duration)
l=list(range(0, num_seconds_video + 1, 60))

if(duration - num_seconds_video > 0):
    l.append(duration)
#text = ''
diz={}

if not os.path.exists(chunk_files_dir):
    os.makedirs(chunk_files_dir)
        
for i in range(len(l)-1):
    audio_text = get_audio_text(l[i], l[i + 1], chunk_files_dir, 'cut{}.mp4'.format(i+1))
    ocr_text = get_image_text(cv2.VideoCapture(os.path.join(chunk_files_dir, 'cut{}.mp4'.format(i+1))))
    
    
    #parser = PlaintextParser.from_string(ocr_text, Tokenizer("english"))
    #summarizer = LexRankSummarizer()
    #num_sentences = 10
    #summary = str(summarizer(parser.document, num_sentences))
     #for sentence in summary:
    #print(sentence)
    
    with open(os.path.join(parent_dir, 'Summarised.txt'), mode ='a') as file:
        file.write("Summerised text for duration {}sec - {}sec:".format(l[i], l[i + 1])) 
        file.write("\n")
        file.write(audio_text)
        file.write("\n")
        file.write(ocr_text)
        file.write("\n\n")
        


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\converted\converted1.wav


MoviePy - Done.
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\0.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\25.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\50.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\75.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\100.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\125.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\150.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\175.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\200.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\225.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\250.png

MoviePy - Done.
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\0.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\25.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\50.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\75.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\100.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\125.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\150.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\175.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\200.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\225.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\250.png

MoviePy - Done.
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\0.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\25.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\50.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\75.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\100.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\125.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\150.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\175.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\200.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\225.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\250.png

MoviePy - Done.
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\0.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\25.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\50.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\75.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\100.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\125.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\150.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\175.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\200.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\225.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\250.png

MoviePy - Done.
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\0.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\25.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\50.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\75.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\100.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\125.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\150.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\175.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\200.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\225.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\250.png

MoviePy - Done.


Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\0.png
Extracting frames...C:\Users\user.DESKTOP-3R3KUAJ\Desktop\Python\nlp\image_frames\25.png


In [26]:
remove_duplicate_sentences('C:\\Users\\user.DESKTOP-3R3KUAJ\\Desktop\\Python\\nlp\\Summarised.txt')

["2\nbe\n\n2\n\nares\n\n© ones\n\n\n\nSummerised text for duration 60sec - 120sec:\nvideo Kiya Epic start this file here here self-extractor here is not supporting operation here that we can be patient different files for example here using local disk c g the local listings that we create Advanced Java so here this is holiday pay I click on this by extract the file here jarvics packages that I said what is solid packaged in a JavaScript select package right I will write the three classes in please observe here so here we define a java\nJava servlet motorcycle welfare circulated generic cialis generic servlet in Java to the selected at it contains http basic Devi contains what is HTTP servlet 3 so fast approach to write my servlet class Mysore let it implements interface implements interface as one here class my soul 8th class my servlet so sweet exchange then the solid solid I didn't the package name S is concentration and packaging\n\n\nSummerised text for duration 120sec - 180sec:\nv